# Machine Learning with Python

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from sklearn.datasets import load_files

reviews_train = load_files("imdb/train/")
text_train, y_train = reviews_train.data, reviews_train.target
text_train = [doc.replace(b"<br />", b" ") for doc in text_train]

reviews_test = load_files("imdb/test/")
text_test, y_test = reviews_test.data, reviews_test.target
text_test = [doc.replace(b"<br />", b" ") for doc in text_test]

## 3.3 Unsupervised Approaches

There are a number of unsupervised approaches in NLP which can give us useful information about the substructure of a corpus, or suggest relevance rankings.

### Topic Modelling

The idea of *topic modelling* is to assign each document to one or more topic classes. For example, news data can usually be assigned to *politics*, *sport*, *finance*, etc.

The *Latent Dirichlet Allocation* (LDA) approach models each document as a weighted *mixture* of topics. Topics will be derived from groups of words that tend to occur together. For example, "MP", "minister" and "vote" will tend to be found together, while "goal", "team" and "score" will tend to be found together. Because this is an unsupervised approach, the topics obtained may not coincide with our understanding of the relevant groups withing the corpus. However, LDA can still be a useful way to reduce the dimensionality of the data for further exploration.


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(max_features=10000, max_df=.15)
X = vect.fit_transform(text_train)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=10,learning_method="batch",
                                max_iter=25, random_state=0)
# We build the model and transform the data in one step
# Computing transform takes some time,
# and we can save time by doing both at once
document_topics = lda.fit_transform(X)

In [ ]:
print("lda.components_.shape: {}".format(lda.components_.shape))

In [ ]:
# for each topic (a row in the components_), sort the features (ascending).
# Invert rows with [:, ::-1] to make sorting descending
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
# get the feature names from the vectorizer:
feature_names = np.array(vect.get_feature_names())

Here are the top-weighted features for the ten topics identified:

In [ ]:
for topic_id in range(len(sorting)):
    features = feature_names[sorting[topic_id]]
    print( "topic", topic_id, features[0:5] )


We can use a larger number of topics to get finer resolution on the substructure of the corpus.

If few training data are available, transforming into the lower-dimensional LDA feature space may be a helpful preprocessing step for supervised learning.


In [ ]:
# sort by weight of "horror" topic 7
horror = np.argsort(document_topics[:, 7])[::-1]
# print the five documents where the topic is most important
for i in horror[:5]:
    # show first two sentences
    print(b".".join(text_train[i].split(b".")[:2]) + b".\n")

### Finding Similar Documents

We can imagine using a distance metric within the *bag of words*, *n-grams* or LDA feature space to find documents that are "similar" to a given input document.

However, these features are not good at capturing the *semantic* similarity between documents. Consider

* The cat sat on the mat
* The feline rested on the rug


`word2vec` is a very popular approach to this problem that is based on neural networks. Using a large unannotated plain text corpus, `word2vec` constructs a *continuous* feature space with a predefined dimensionality, within which individual words are located. Words with similar semantic meaning are located close together in this feature space. More remarkably, certain linear relationships can be identified:

vec(“king”) - vec(“man”) + vec(“woman”) =~ vec(“queen”)

We can use the `gensim` package to explore some of the possibilities of this approach.

https://radimrehurek.com/gensim/index.html

Firstly, we will download a pre-trained model with 100 dimensions, trained on 400,000 Wikipedia articles from 2014.

In [ ]:
import gensim.downloader as api

model = api.load("glove-wiki-gigaword-100")


In [ ]:
model

The model gives the vector representation of any word from the corpus:

In [ ]:
model['potato']

In [ ]:
model['potato'].shape

The model can compute similarities to other words.

In [ ]:
model.most_similar("potato")

By embedding documents into this space, document similarity can be computed.